In [1]:
from uncertainties import measurement as ms
import pandas as pd
from sympy import latex
data = pd.read_csv('data.csv')
k = ms.Observable(symbol='k', definition='m*(T - T_e)/(T_e - T_0) - M', data=data, last_row_error=True)
k.calc_error(method='derivatives')
k.calc_values()
k.report(return_tex=False, generate_pdf=True)

In [2]:
# TODO test now the rest of the experiment
# TODO next step is allow *variables* to be *constants* when evaluated, meaning that they do not change after every take
# TODO in this example, k is a characteristic of the calorimeter 

In [78]:
function_vars = latex(k.symbol) + ' = ' + latex(k.symbol) + latex(tuple(k.definition.free_symbols))
expression = latex(k.symbol) + ' = ' + latex(k.definition)
data = k.data.copy(deep=True)

data[k.symbol.name] = k.values
data.columns = ['$' + var_name + '$' for var_name in data.columns]
data.index += 1
data = data.style.format('{:.2f}').to_latex()

reported_value = r'\bar ' + latex(k.symbol) + ' = ' + str(k.mean_value) + r'\pm' + str(k.error)
summary_items = (function_vars, expression, data, reported_value)
summary_titles = ('Variables', 'Definición matemática', 'Datos tomados', 'Valor reportado')


In [2]:

# Basic document
doc = Document('report')
doc.preamble.append(Command('title', 'Summary of Measurement'))
doc.preamble.append(Command('author', 'Grupo 1'))
doc.preamble.append(Command('date', NoEscape(r'\today')))
doc.append(NoEscape(r'\maketitle'))
for title, item in zip(summary_titles, summary_items):
    with doc.create(Section(title)):
        doc.append(Math(data=item, escape=False))
doc.generate_pdf(clean_tex=True)
tex = doc.dumps()  # The document as string in LaTeX syntax


NameError: name 'Document' is not defined

In [3]:
print(tex)

NameError: name 'tex' is not defined